In [1]:
from selenium.common.exceptions import NoSuchElementException, ElementClickInterceptedException
from selenium import webdriver
import time
import pandas as pd

In [4]:
pip install selenium

Note: you may need to restart the kernel to use updated packages.


In [2]:
def get_jobs(keyword, num_jobs, verbose):
    
    '''Gathers jobs as a dataframe, scraped from Glassdoor'''
    
    #Initializing the webdriver
    options = webdriver.ChromeOptions()
    
    #Uncomment the line below if you'd like to scrape without a new Chrome window every time.
    #options.add_argument('headless')
    
    #Change the path to where chromedriver is in your home folder.
    driver = webdriver.Chrome(executable_path="/Users/User/Downloads/chromedriver", options=options)
    driver.set_window_size(1120, 1000)
    url = 'https://www.glassdoor.com/Job/jobs.htm?sc.keyword="' + keyword + '"&locT=C&locId=1147401&locKeyword=San%20Francisco,%20CA&jobType=all&fromAge=-1&minSalary=0&includeNoSalaryJobs=true&radius=100&cityId=-1&minRating=0.0&industryId=-1&sgocId=-1&seniorityType=all&companyId=-1&employerSizes=0&applicationType=0&remoteWorkType=0'
    driver.get(url)
    jobs = []

    while len(jobs) < num_jobs:  #If true, should be still looking for new jobs.

        #Let the page load. Change this number based on your internet speed.
        #Or, wait until the webpage is loaded, instead of hardcoding it.
        time.sleep(4)

        #Test for the "Sign Up" prompt and get rid of it.
        try:
            driver.find_element_by_class_name("selected").click()
        except ElementClickInterceptedException:
            pass

        time.sleep(.1)

        try:
            driver.find_element_by_class_name("ModalStyle__xBtn___29PT9").click()  #clicking to the X.
        except NoSuchElementException:
            pass

        
        #Going through each job in this page
        job_buttons = driver.find_elements_by_class_name("jl")  #jl for Job Listing. These are the buttons we're going to click.
        for job_button in job_buttons:  

            print("Progress: {}".format("" + str(len(jobs)) + "/" + str(num_jobs)))
            if len(jobs) >= num_jobs:
                break

            job_button.click()  #You might 
            time.sleep(1)
            collected_successfully = False
            
            while not collected_successfully:
                try:
                    company_name = driver.find_element_by_xpath('.//div[@class="employerName"]').text
                    location = driver.find_element_by_xpath('.//div[@class="location"]').text
                    job_title = driver.find_element_by_xpath('.//div[contains(@class, "title")]').text
                    job_description = driver.find_element_by_xpath('.//div[@class="jobDescriptionContent desc"]').text
                    collected_successfully = True
                except:
                    time.sleep(5)

            try:
                salary_estimate = driver.find_element_by_xpath('.//span[@class="gray small salary"]').text
            except NoSuchElementException:
                salary_estimate = -1 #You need to set a "not found value. It's important."
            
            try:
                rating = driver.find_element_by_xpath('.//span[@class="rating"]').text
            except NoSuchElementException:
                rating = -1 #You need to set a "not found value. It's important."

            #Printing for debugging
            if verbose:
                print("Job Title: {}".format(job_title))
                print("Salary Estimate: {}".format(salary_estimate))
                print("Job Description: {}".format(job_description[:500]))
                print("Rating: {}".format(rating))
                print("Company Name: {}".format(company_name))
                print("Location: {}".format(location))

            #Going to the Company tab...
            #clicking on this:
            #<div class="tab" data-tab-type="overview"><span>Company</span></div>
            try:
                driver.find_element_by_xpath('.//div[@class="tab" and @data-tab-type="overview"]').click()

                try:
                    #<div class="infoEntity">
                    #    <label>Headquarters</label>
                    #    <span class="value">San Francisco, CA</span>
                    #</div>
                    headquarters = driver.find_element_by_xpath('.//div[@class="infoEntity"]//label[text()="Headquarters"]//following-sibling::*').text
                except NoSuchElementException:
                    headquarters = -1

                try:
                    size = driver.find_element_by_xpath('.//div[@class="infoEntity"]//label[text()="Size"]//following-sibling::*').text
                except NoSuchElementException:
                    size = -1

                try:
                    founded = driver.find_element_by_xpath('.//div[@class="infoEntity"]//label[text()="Founded"]//following-sibling::*').text
                except NoSuchElementException:
                    founded = -1

                try:
                    type_of_ownership = driver.find_element_by_xpath('.//div[@class="infoEntity"]//label[text()="Type"]//following-sibling::*').text
                except NoSuchElementException:
                    type_of_ownership = -1

                try:
                    industry = driver.find_element_by_xpath('.//div[@class="infoEntity"]//label[text()="Industry"]//following-sibling::*').text
                except NoSuchElementException:
                    industry = -1

                try:
                    sector = driver.find_element_by_xpath('.//div[@class="infoEntity"]//label[text()="Sector"]//following-sibling::*').text
                except NoSuchElementException:
                    sector = -1

                try:
                    revenue = driver.find_element_by_xpath('.//div[@class="infoEntity"]//label[text()="Revenue"]//following-sibling::*').text
                except NoSuchElementException:
                    revenue = -1

                try:
                    competitors = driver.find_element_by_xpath('.//div[@class="infoEntity"]//label[text()="Competitors"]//following-sibling::*').text
                except NoSuchElementException:
                    competitors = -1

            except NoSuchElementException:  #Rarely, some job postings do not have the "Company" tab.
                headquarters = -1
                size = -1
                founded = -1
                type_of_ownership = -1
                industry = -1
                sector = -1
                revenue = -1
                competitors = -1

                
            if verbose:
                print("Headquarters: {}".format(headquarters))
                print("Size: {}".format(size))
                print("Founded: {}".format(founded))
                print("Type of Ownership: {}".format(type_of_ownership))
                print("Industry: {}".format(industry))
                print("Sector: {}".format(sector))
                print("Revenue: {}".format(revenue))
                print("Competitors: {}".format(competitors))
                print("@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@")

            jobs.append({"Job Title" : job_title,
            "Salary Estimate" : salary_estimate,
            "Job Description" : job_description,
            "Rating" : rating,
            "Company Name" : company_name,
            "Location" : location,
            "Headquarters" : headquarters,
            "Size" : size,
            "Founded" : founded,
            "Type of ownership" : type_of_ownership,
            "Industry" : industry,
            "Sector" : sector,
            "Revenue" : revenue,
            "Competitors" : competitors})
            #add job to jobs

        #Clicking on the "next page" button
        try:
            driver.find_element_by_xpath('.//li[@class="next"]//a').click()
        except NoSuchElementException:
            print("Scraping terminated before reaching target number of jobs. Needed {}, got {}.".format(num_jobs, len(jobs)))
            break

    return pd.DataFrame(jobs)  #This line converts the dictionary object into a pandas DataFrame.


In [8]:
df = get_jobs("web developer",500, False)
df

Progress: 0/500
Progress: 1/500
Progress: 2/500
Progress: 3/500
Progress: 4/500
Progress: 5/500
Progress: 6/500
Progress: 7/500
Progress: 8/500
Progress: 9/500
Progress: 10/500
Progress: 11/500
Progress: 12/500
Progress: 13/500
Progress: 14/500
Progress: 15/500
Progress: 16/500
Progress: 17/500
Progress: 18/500
Progress: 19/500
Progress: 20/500
Progress: 21/500
Progress: 22/500
Progress: 23/500
Progress: 24/500
Progress: 25/500
Progress: 26/500
Progress: 27/500
Progress: 28/500
Progress: 29/500
Progress: 30/500
Progress: 31/500
Progress: 32/500
Progress: 33/500
Progress: 34/500
Progress: 35/500
Progress: 36/500
Progress: 37/500
Progress: 38/500
Progress: 39/500
Progress: 40/500
Progress: 41/500
Progress: 42/500
Progress: 43/500
Progress: 44/500
Progress: 45/500
Progress: 46/500
Progress: 47/500
Progress: 48/500
Progress: 49/500
Progress: 50/500
Progress: 51/500
Progress: 52/500
Progress: 53/500
Progress: 54/500
Progress: 55/500
Progress: 56/500
Progress: 57/500
Progress: 58/500
Progres

,Company Name,Competitors,Founded,Headquarters,Industry,Job Description,Job Title,Location,Rating,Revenue,Salary Estimate,Sector,Size,Type of ownership
0,Acumen\n3.3,Acumen,1996,"Burlingame, CA",Government Agencies,Job Description\n\nWe are looking for a full-s...,Software and Web Developer I,"San Francisco, CA",3.3,₹1 to ₹5 billion (INR),-1,Government,201 to 500 employees,Company - Private
1,TriNet\n3.3,"Paychex, Insperity, ADP",1988,"Dublin, CA",Consulting,TriNet is a leading provider of comprehensive ...,Front End Web Developer,"Dublin, CA",3.3,₹100 to ₹500 billion (INR),-1,Business Services,1001 to 5000 employees,Company - Public
2,Adobe\n4.0,"Apple, Microsoft",1982,"San Jose, CA",Computer Hardware & Software,The Challenge\nAdobe Document Cloud is revolut...,"Full Stack Web Developer, Document Cloud Web E...","San Jose, CA",4.0,₹500+ billion (INR),-1,Information Technology,10000+ employees,Company - Public
3,Western Digital\n3.5,"Seagate Technology, Toshiba",1970,"San Jose, CA",Computer Hardware & Software,Western Digital\nThe next big thing in data is...,Full Stack Web Developer,"Milpitas, CA",3.5,₹500+ billion (INR),-1,Information Technology,10000+ employees,Company - Public
4,EandM\n4.5,-1,-1,"Healdsburg, CA",-1,Job Description:\n\nEandM is expanding and is ...,.Net Web Developer,"Healdsburg, CA",4.5,Unknown / Non-Applicable,-1,-1,51 to 200 employees,Company - Private
5,Ouster\n4.2,-1,2015,"San Francisco, CA",Electrical & Electronic Manufacturing,"Come Build Robots\n\n\nAt Ouster, we build sen...",Front-End Web Developer,"San Francisco, CA",4.2,Unknown / Non-Applicable,-1,Manufacturing,51 to 200 employees,Company - Private
6,Adobe\n4.0,"Apple, Microsoft",1982,"San Jose, CA",Computer Hardware & Software,Our Company\n\n\nChanging the world through di...,Web Developer Intern,"San Francisco, CA",4.0,₹500+ billion (INR),-1,Information Technology,10000+ employees,Company - Public
7,Dolls Kill\n3.6,-1,2011,"San Francisco, CA",Other Retail Shops,"Dolls Kill, named “The Fastest Growing Private...",Web Developer (Junior & Senior),"San Francisco, CA",3.6,₹100 to ₹500 million (INR),-1,Retail,201 to 500 employees,Company - Private
8,StudySoup\n4.0,-1,2014,"San Francisco, CA",Colleges & Universities,This position is the driving force behind Stud...,Web Developer,"San Francisco, CA",4.0,Unknown / Non-Applicable,-1,Education,1 to 50 employees,Company - Private
9,Flexton\n4.0,-1,2007,"San Jose, CA",IT Services,Job TITLE: Web Developer\n\nLocation: San Brun...,Web Developer,"San Bruno, CA",4.0,₹1 to ₹5 billion (INR),-1,Information Technology,51 to 200 employees,Company - Private


In [9]:
df.to_excel(r'D:\web_developer_output.xlsx', index = False)